In [2]:
# necessary libraries for prediciton
import utils
import pandas as pd
import numpy as np
import os

# Prediction

* Predictions are done in this notebook.
* It is also useful to compare how serveral algorithms perform against one another.

In [44]:
dataset =  utils.read_csv_to_df('dataset/preprocessed_data.csv')

In [14]:
# Useful MACROS
STATUS_COL = 5

In [42]:
# Setting X and Y
# X = dataset.iloc[:, 0:STATUS_COL]
# y = dataset.iloc[:, STATUS_COL]

In [51]:
display(dataset[[0]])

KeyError: "None of [Int64Index([0], dtype='int64')] are in the [columns]"

FileNotFoundError: [Errno 2] File b'housing.csv' does not exist: b'housing.csv'